In [14]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from datetime import datetime
import glob
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Selecting the GEFS data and comparing them with ERA5 observations. Choose either RWP envelope or phase speed. Only select Northern Hemisphere

In [15]:
path = '/media/onno/Algemeen/Thesis/'
# file_gfs = 'gefsrf2_env_wledit2000-10000_latavg_v300_control0-252h_6hourly_2x2_dec84-nov19.nc'
# file_era = 'era51_env_wledit2000-10000_latavg_v300_79-19_6hourly_anom_from_smoothed04_clim_smoothed.nc'
# file_gfs = 'gefsrf2_phasevel_wledit2000-10000_latavg_v300_envgt15_control0-252h_6hourly_2x2_dec84-nov19_setvrange_-100to100.nc'
# file_era = 'era51_phasevel_wledit2000-10000_latavg_v300_envgt15_79-19_6hourly_anom_from_smoothed04_clim_setvrange_-100to100.nc'
file_gfs = 'gefsrf2_u300_control0-252h_6hourly_2x2_dec84-nov19.nc'
file_era = 'era5_u300_79-19_6hourly.nc'
gfs = xr.open_dataset(path+file_gfs,decode_times=False)
era = xr.open_dataset(path+file_era,decode_times=False)
#Convert dates to standard datetime values
gfs['time']=my_tools.convert_date_gefs_r(gfs.time.values)
era['time']=my_tools.convert_date_era_r(era.time.values)

In [16]:
gfs_nh = gfs.sel(lat=slice(90,0),lead=gfs.lead.values[[bool(i) for i in gfs.lead%24==0]])
era_nh = era.sel(lat=slice(90,0))

Calculating daily mean errors 

In [ ]:

for i in range(11):
    #Select days for which GEFS data is present and make datasets equally long
    gfs_nh_d = gfs_nh.sel(time=[i for i in pd.date_range('1984-12-1','2019-11-{}'.format(30-i))],lead=i*24)
    era_nh_d = era_nh.sel(time=[i for i in pd.date_range('1984-12-{}'.format(1+i),'2019-11-30')]) 
    print(i)
    #Make boolean array for each day that is present in season        
    #Check if dates are correct
    assert(gfs_nh_d.time.values[0]==(era_nh_d.time.values[0]-pd.Timedelta(i,'d')))
    gfs_season_mean = gfs_nh_d.mean(dim='time')
    era_season_mean = era_nh_d.mean(dim='time')
    diff = gfs_season_mean - era_season_mean
    diff.to_netcdf('/media/onno/Algemeen/Thesis/GFS_mean/GFS_mean_error_u_wind_day_{:02d}.nc'.format(i))


0


In [6]:
t1 = datetime.now()
for i in range(1,11):
    forecast = gfs_nh.sel(time=[my_tools.convert_date_gefs(i) for i in pd.date_range('2014-12-1','2019-11-{}'.format(30-i))],lead=i*24)
    reanalysis = era_nh.sel(time=[my_tools.convert_date_era(i) for i in pd.date_range('2014-12-{}'.format(1+i),'2019-11-30')])
    assert(my_tools.convert_date_gefs_r(float(forecast.time[0]+forecast.lead.values))==my_tools.convert_date_era_r(float(reanalysis.time[0])))
    reanalysis['time'] = forecast.time.values
#     forecast_mean = forecast.mean(dim='time')
#     reanalysis_mean = reanalysis.mean(dim='time')
    diff2 = forecast - reanalysis
    diff2 = diff2.mean(dim='time')
    break
    diff.to_netcdf('/media/onno/Algemeen/Thesis/GFS_mean/bias_GFS_forecast_phasespeed_day{:02d}.nc'.format(i))
    print(i)
t2 = datetime.now()
print(t2-t1)


0:00:01.863019


1708584.0